In [1]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
 
################################################
# PLUS 공통 OBJECT
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
 
 
################################################
# PLUS 실행 기본 체크 함수
def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # # 주문 관련 초기화 - 계좌 관련 코드가 있을 때만 사용
    # if (g_objCpTrade.TradeInit(0) != 0):
    #     print("주문 초기화 실패")
    #     return False
 
    return True
 

In [2]:
 
class CpMarketEye:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("CpSysDib.MarketEye")
        self.RpFiledIndex = 0
 
 
    def Request(self, codes, dataInfo):
        # 0: 종목코드 4: 현재가 20: 상장주식수
        rqField = [0, 4, 20]  # 요청 필드
 
        self.objRq.SetInputValue(0, rqField)  # 요청 필드
        self.objRq.SetInputValue(1, codes)  # 종목코드 or 종목코드 리스트
        self.objRq.BlockRequest()
 
        # 현재가 통신 및 통신 에러 처리
        rqStatus = self.objRq.GetDibStatus()
        print("통신상태", rqStatus, self.objRq.GetDibMsg1())
        if rqStatus != 0:
            return False
 
        cnt = self.objRq.GetHeaderValue(2)
 
        for i in range(cnt):
            code = self.objRq.GetDataValue(0, i)  # 코드
            cur = self.objRq.GetDataValue(1, i)  # 종가
            listedStock = self.objRq.GetDataValue(2, i)  # 상장주식수
 
            maketAmt = listedStock * cur
            if g_objCodeMgr.IsBigListingStock(code) :
                maketAmt *= 1000
#            print(code, maketAmt)
 
            # key(종목코드) = tuple(상장주식수, 시가총액)
            dataInfo[code] = (listedStock, maketAmt)
 
        return True
 
class CMarketTotal():
    def __init__(self):
        self.dataInfo = {}
 
 
    def GetAllMarketTotal(self):
        codeList = g_objCodeMgr.GetStockListByMarket(1)  # 거래소
        codeList2 = g_objCodeMgr.GetStockListByMarket(2)  # 코스닥
        allcodelist = codeList + codeList2
        print('전 종목 코드 %d, 거래소 %d, 코스닥 %d' % (len(allcodelist), len(codeList), len(codeList2)))
 
        objMarket = CpMarketEye()
        rqCodeList = []
        for i, code in enumerate(allcodelist):
            rqCodeList.append(code)
            if len(rqCodeList) == 200:
                objMarket.Request(rqCodeList, self.dataInfo)
                rqCodeList = []
                continue
        # end of for
 
        if len(rqCodeList) > 0:
            objMarket.Request(rqCodeList, self.dataInfo)
 
    def PrintMarketTotal(self):
 
        # 시가총액 순으로 소팅
        data2 = sorted(self.dataInfo.items(), key=lambda x: x[1][1], reverse=True)
 
        print('전종목 시가총액 순 조회 (%d 종목)' % (len(data2)))
        for item in data2:
            name = g_objCodeMgr.CodeToName(item[0])
            listed = item[1][0]
            markettot = item[1][1]
            print('%s 상장주식수: %s, 시가총액 %s' %(name, format(listed, ','), format(markettot, ',')))
 

In [6]:
objMarketTotal = CMarketTotal()
objMarketTotal.GetAllMarketTotal()
objMarketTotal.PrintMarketTotal()

전 종목 코드 2896, 거래소 1530, 코스닥 1366
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
통신상태 0 0027 조회가 완료되었습니다.(market.eye)
전종목 시가총액 순 조회 (2896 종목)
삼성전자 상장주식수: 5,969,782, 시가총액 265,356,809,900,000
SK하이닉스 상장주식수: 728,002,365, 시가총액 55,619,380,686,000
삼성전자우 상장주식수: 822,886,700, 시가총액 30,487,952,235,000
현대차 상장주식수: 213,668,187, 시가총액 27,029,025,655,500
NAVER 상장주식수: 164,813,395, 시가총액 23,980,348,972,500
현대모비스 상장주식수: 95,306,694, 시가총액 23,302,486,683,000
LG화학 상장주식수: 70,592,343, 시가총액 22,871,919,132,000
셀트리온 상장주식수: 128,336,328, 시가총액 19,892,130,840

이트론 상장주식수: 349,598,748, 시가총액 145,433,079,168
남성 상장주식수: 36,212,160, 시가총액 145,210,761,600
KG케미칼 상장주식수: 13,013,232, 시가총액 145,097,536,800
액토즈소프트 상장주식수: 11,330,638, 시가총액 145,032,166,400
노루페인트 상장주식수: 20,000,000, 시가총액 145,000,000,000
NPC 상장주식수: 36,720,000, 시가총액 144,309,600,000
UCI 상장주식수: 33,091,055, 시가총액 143,780,633,975
쎄트렉아이 상장주식수: 7,268,487, 시가총액 143,552,618,250
대성에너지 상장주식수: 27,500,000, 시가총액 143,550,000,000
남화산업 상장주식수: 20,588,000, 시가총액 143,498,360,000
레드캡투어 상장주식수: 8,589,480, 시가총액 143,014,842,000
피앤이솔루션 상장주식수: 14,280,000, 시가총액 142,800,000,000
쌍방울 상장주식수: 129,124,213, 시가총액 142,682,255,365
에이디테크놀로지 상장주식수: 9,261,533, 시가총액 142,627,608,200
삼보판지 상장주식수: 16,170,000, 시가총액 142,619,400,000
메카로 상장주식수: 10,074,640, 시가총액 142,052,424,000
BYC 상장주식수: 624,615, 시가총액 141,787,605,000
SK이노베이션우 상장주식수: 1,248,426, 시가총액 141,696,351,000
핸즈코퍼레이션 상장주식수: 21,863,268, 시가총액 141,673,976,640
코리아나 상장주식수: 40,000,000, 시가총액 141,600,000,000
탑엔지니어링 상장주식수: 15,976,257, 시가총액 141,549,637,020
삼목에스폼 상장주식수: 14,700,000, 시가총액 141,414,000,000


대주산업 상장주식수: 35,392,350, 시가총액 46,010,055,000
파인텍 상장주식수: 42,921,361, 시가총액 45,925,856,270
시스웍 상장주식수: 18,000,000, 시가총액 45,810,000,000
이노테라피 상장주식수: 5,044,184, 시가총액 45,801,190,720
한국특수형강 상장주식수: 32,353,490, 시가총액 45,780,188,350
엑시콘 상장주식수: 8,883,500, 시가총액 45,750,025,000
서진오토모티브 상장주식수: 19,541,246, 시가총액 45,726,515,640
케이엠제약 상장주식수: 21,387,050, 시가총액 45,661,351,750
MP한강 상장주식수: 56,831,270, 시가총액 45,635,509,810
엔에스 상장주식수: 4,900,000, 시가총액 45,472,000,000
모헨즈 상장주식수: 10,920,000, 시가총액 45,427,200,000
네오티스 상장주식수: 11,013,678, 시가총액 45,376,353,360
SGA솔루션즈 상장주식수: 36,676,033, 시가총액 45,111,520,590
한솔시큐어 상장주식수: 7,366,074, 시가총액 45,080,372,880
케이씨에스 상장주식수: 12,000,000, 시가총액 45,060,000,000
팬엔터테인먼트 상장주식수: 13,742,997, 시가총액 45,008,315,175
흥아해운 상장주식수: 97,415,733, 시가총액 44,908,652,913
SJM홀딩스 상장주식수: 14,934,008, 시가총액 44,876,694,040
영화테크 상장주식수: 5,345,090, 시가총액 44,738,403,300
그린플러스 상장주식수: 3,923,873, 시가총액 44,732,152,200
아이컴포넌트 상장주식수: 7,070,860, 시가총액 44,687,835,200
바다로19호 상장주식수: 15,200,004, 시가총액 44,612,011,740
TJ미디어 상장주식수: 13,931,60

In [10]:
data2 = sorted(objMarketTotal.dataInfo.items(), key=lambda x: x[1][1], reverse=True)

In [11]:
MarketTotal = []

for item in data2:
    name = g_objCodeMgr.CodeToName(item[0])
    listed = item[1][0]
    markettot = item[1][1]
    
    MarketTotal.append([name, item[0], listed, markettot])

In [12]:
import pandas as pd

In [13]:
pdMarketTotal = pd.DataFrame(MarketTotal)

In [15]:
pdMarketTotal.head()

,0,1,2,3
0,삼성전자,A005930,5969782,265356809900000
1,SK하이닉스,A000660,728002365,55619380686000
2,삼성전자우,A005935,822886700,30487952235000
3,현대차,A005380,213668187,27029025655500
4,NAVER,A035420,164813395,23980348972500


In [17]:
pdMarketTotal[pdMarketTotal[0].apply(lambda x: 'KODEX' in x)]

,0,1,2,3
44,KODEX 200,A069500,219250000,5662131250000
93,KODEX 레버리지,A122630,222200000,2426424000000
143,KODEX 단기채권,A153130,13615000,1379131425000
173,KODEX 코스닥150 레버리지,A233740,168700000,1104985000000
176,KODEX 종합채권(AA-이상)액티브,A273130,9898000,1086602440000
190,KODEX 삼성그룹,A102780,162100000,987189000000
194,KODEX 200TR,A278530,115800000,937980000000
204,KODEX 단기채권PLUS,A214980,8822000,894065590000
224,KODEX MSCI Korea TR,A278540,97400000,780661000000
250,KODEX Top5PlusTR,A315930,57700000,706825000000


In [13]:
pdMarketTotal.columns = "Name,Code,Stock,capitalization".split(",")

In [15]:
pdMarketTotal.to_csv("../data/market_capitalization.csv", index=None)